In [125]:
import networkx as nx
import matplotlib.pyplot as plt
import pydot
from pydot import Edge
from pydot import Node
import re

global subgraphs, nodes_subgraphs
subgraphs = {}
nodes_subgraphs = {}

In [126]:
def get_graph_from_dot(file):
    """
       Import a graph from dot and return the pydot graph
       
       :param file: path to the file to import
       :return: pydot graph
       :rtype: Dot
    """
    (graph, ) = pydot.graph_from_dot_file(str(file))
    return graph

graph = get_graph_from_dot('./main_output.dot')

In [127]:
def extract_subgraphs(subgraph_list):
    global subgraphs
    """
       Extract subgraphs recursively from a list of subgraphs, and the hierarchy of the subgraphs
       
       :param graph: graph or subgraph from which subgraphs are to be extracted
       :return: List of subgraphs
       :rtype: List
    """
    new_subgraphs = []
    for subgraph in subgraph_list:
        subgraph.set_name(subgraph.get_name().strip("\"").replace("cluster_", ""))
        subgraphs[subgraph.get_name()] = subgraph
        children_subgraphs = subgraph.get_subgraph_list() 
        new_subgraphs += children_subgraphs
    if(new_subgraphs == []):
        return
    else:
        return extract_subgraphs(new_subgraphs)

extract_subgraphs([graph])

In [128]:
def associate_node_subgraph (graph):
    global nodes_subgraphs, subgraphs
    
    for key, subgraph in subgraphs.items():
        if(type(subgraph) == pydot.Subgraph):
            for node in subgraph.get_nodes():
                if(node.get_name() != 'graph'):
                    nodes_subgraphs[node.get_name()] = subgraph

associate_node_subgraph(graph)
print(nodes_subgraphs)

{'fn_0_basic_block_0': <pydot.Subgraph object at 0x7fae0b36cac8>, 'fn_0_basic_block_1': <pydot.Subgraph object at 0x7fae0b36cac8>, 'fn_0_basic_block_2': <pydot.Subgraph object at 0x7fae0b36cac8>, 'fn_0_basic_block_3': <pydot.Subgraph object at 0x7fae0b36cac8>, 'fn_0_basic_block_4': <pydot.Subgraph object at 0x7fae0b36cac8>, 'fn_0_basic_block_5': <pydot.Subgraph object at 0x7fae0b36cac8>, 'fn_1_basic_block_0': <pydot.Subgraph object at 0x7fae0b36c3c8>, 'fn_1_basic_block_1': <pydot.Subgraph object at 0x7fae0b36c3c8>, 'fn_1_basic_block_2': <pydot.Subgraph object at 0x7fae0b36c3c8>, 'fn_1_basic_block_3': <pydot.Subgraph object at 0x7fae0b36c3c8>, 'fn_2_basic_block_0': <pydot.Subgraph object at 0x7fae0b36c0f0>, 'fn_2_basic_block_1': <pydot.Subgraph object at 0x7fae0b36c0f0>, 'fn_2_basic_block_2': <pydot.Subgraph object at 0x7fae0b36c0f0>, 'fn_2_basic_block_5': <pydot.Subgraph object at 0x7fae0b36c0f0>, 'fn_2_basic_block_7': <pydot.Subgraph object at 0x7fae0b36c0f0>, 'fn_2_basic_block_6': <p

In [142]:
def find_number_in_regex(regex, string):
    return re.search(r'\d+', re.search(regex, string).group()).group()

print(find_number_in_regex(r'yolo_\d+', "j'aime le yolo_34 zqfzef"))

34


In [130]:
def move_edges (graph_name):
    global subgraphs
    
    for key, subgraph in subgraphs.items():
        for edge in subgraph.get_edges():
            node_head = edge.get_source()
            node_tail = edge.get_destination()
            subgraph.del_edge(node_head, node_tail, 0)
            if(find_number_in_regex(r'block_\d+', node_head) != "0" 
                or find_number_in_regex(r'block_\d+', node_tail) != "1"):
                print(node_head + " " + node_tail)
                subgraphs[graph_name].add_edge(Edge(node_head[:-2], node_tail[:-2]))
            
move_edges(graph.get_name())
print(graph.get_edges())

fn_0_basic_block_1:s fn_2_basic_block_4:n
fn_1_basic_block_1:s fn_2_basic_block_3:n
fn_2_basic_block_2:s fn_1_basic_block_0:n
fn_2_basic_block_3:s fn_0_basic_block_0:n
fn_0_basic_block_0:s fn_0_basic_block_2:n
fn_0_basic_block_2:s fn_0_basic_block_3:n
fn_0_basic_block_2:s fn_0_basic_block_4:n
fn_0_basic_block_3:s fn_0_basic_block_5:n
fn_0_basic_block_4:s fn_0_basic_block_5:n
fn_0_basic_block_5:s fn_0_basic_block_1:n
fn_1_basic_block_0:s fn_1_basic_block_2:n
fn_1_basic_block_2:s fn_1_basic_block_3:n
fn_1_basic_block_3:s fn_1_basic_block_1:n
fn_2_basic_block_0:s fn_2_basic_block_2:n
fn_2_basic_block_5:s fn_2_basic_block_7:n
fn_2_basic_block_6:s fn_2_basic_block_7:n
fn_2_basic_block_7:s fn_2_basic_block_8:n
fn_2_basic_block_8:s fn_2_basic_block_1:n
fn_2_basic_block_4:s fn_2_basic_block_5:n
fn_2_basic_block_4:s fn_2_basic_block_6:n
[<pydot.Edge object at 0x7fae0b342f60>, <pydot.Edge object at 0x7fae0b342f98>, <pydot.Edge object at 0x7fae0b2c60b8>, <pydot.Edge object at 0x7fae0b342128>, <py

In [131]:
def get_networkx_graph(graph):
    G = nx.drawing.nx_pydot.from_pydot(graph)
    nx.freeze(G)
    return G

G = get_networkx_graph(graph)
print(G)

test.c.011t.cfg


In [132]:
for node in G.nodes():
    print(node)

fn_0_basic_block_1
fn_2_basic_block_4
fn_1_basic_block_1
fn_2_basic_block_3
fn_2_basic_block_2
fn_1_basic_block_0
fn_0_basic_block_0
fn_0_basic_block_2
fn_0_basic_block_3
fn_0_basic_block_4
fn_0_basic_block_5
fn_1_basic_block_2
fn_1_basic_block_3
fn_2_basic_block_0
fn_2_basic_block_5
fn_2_basic_block_7
fn_2_basic_block_6
fn_2_basic_block_8
fn_2_basic_block_1


In [133]:
def get_none_in_degree_nodes (G):
    nodes = []
    for (node, degree) in G.in_degree():
        if (degree == 0):
            nodes.append(node)
    return nodes

nodes = get_none_in_degree_nodes(G)
nodes_subgraphs[nodes[0]].get_name()

'main'

In [134]:
def get_betweeness_centrality (G):
    return nx.betweenness_centrality(G)

print(get_betweeness_centrality(G))

{'fn_0_basic_block_1': 0.23529411764705882, 'fn_2_basic_block_4': 0.21241830065359477, 'fn_1_basic_block_1': 0.21241830065359477, 'fn_2_basic_block_3': 0.23529411764705882, 'fn_2_basic_block_2': 0.05555555555555556, 'fn_1_basic_block_0': 0.10457516339869281, 'fn_0_basic_block_0': 0.25163398692810457, 'fn_0_basic_block_2': 0.26143790849673204, 'fn_0_basic_block_3': 0.11764705882352941, 'fn_0_basic_block_4': 0.11764705882352941, 'fn_0_basic_block_5': 0.25163398692810457, 'fn_1_basic_block_2': 0.14705882352941177, 'fn_1_basic_block_3': 0.18300653594771243, 'fn_2_basic_block_0': 0.0, 'fn_2_basic_block_5': 0.06862745098039216, 'fn_2_basic_block_7': 0.10457516339869281, 'fn_2_basic_block_6': 0.06862745098039216, 'fn_2_basic_block_8': 0.05555555555555556, 'fn_2_basic_block_1': 0.0}


In [135]:
def is_acyclic(G):
    return nx.is_directed_acyclic_graph(G)

print(is_acyclic(G))

True


In [136]:
def get_longest_path(G):
    if (is_acyclic(G)):
        return nx.algorithms.dag_longest_path(G)
    raise Exception("Le graphe contient un cycle, il n'a pas de plus long chemin")
    
get_longest_path(G)

['fn_2_basic_block_0',
 'fn_2_basic_block_2',
 'fn_1_basic_block_0',
 'fn_1_basic_block_2',
 'fn_1_basic_block_3',
 'fn_1_basic_block_1',
 'fn_2_basic_block_3',
 'fn_0_basic_block_0',
 'fn_0_basic_block_2',
 'fn_0_basic_block_3',
 'fn_0_basic_block_5',
 'fn_0_basic_block_1',
 'fn_2_basic_block_4',
 'fn_2_basic_block_5',
 'fn_2_basic_block_7',
 'fn_2_basic_block_8',
 'fn_2_basic_block_1']

In [137]:
def get_shortest_path(G):
    global subgraphs
    main_number = find_number_in_regex(r'fn_\d+',subgraphs["main"].get_nodes()[1].get_name())
    source = "fn_" + main_number + "_basic_block_0"
    target = "fn_" + main_number + "_basic_block_1"
    return nx.algorithms.shortest_paths.unweighted.bidirectional_shortest_path(G, source, target)

print(get_shortest_path(G))

['fn_2_basic_block_0', 'fn_2_basic_block_2', 'fn_1_basic_block_0', 'fn_1_basic_block_2', 'fn_1_basic_block_3', 'fn_1_basic_block_1', 'fn_2_basic_block_3', 'fn_0_basic_block_0', 'fn_0_basic_block_2', 'fn_0_basic_block_3', 'fn_0_basic_block_5', 'fn_0_basic_block_1', 'fn_2_basic_block_4', 'fn_2_basic_block_5', 'fn_2_basic_block_7', 'fn_2_basic_block_8', 'fn_2_basic_block_1']


In [140]:
def main (file):
    global subgraphs, nodes_subgraphs
    subgraphs = {}
    nodes_subgraphs = {}
    pydot_graph = get_graph_from_dot(file)
    extract_subgraphs([pydot_graph])
    associate_node_subgraph(pydot_graph)
    move_edges(pydot_graph.get_name())
    nx_graph = get_networkx_graph(pydot_graph)
    
    none_in_degree_nodes = get_none_in_degree_nodes(nx_graph)
    is_acyclic_graph = is_acyclic(nx_graph)
    if (is_acyclic_graph):
        longest_path = get_longest_path(nx_graph)
    shortest_path = get_shortest_path(nx_graph)
    
    
    
    
    for node in none_in_degree_nodes:
        print("Il y a un noeud sans entrée dans la fonction " + nodes_subgraphs[node].get_name() + "\n")
    if (is_acyclic_graph):
        print("Le graphe est acyclique\n")
        print("Le plus long chemin a " + str(len(longest_path)) + " noeuds et est le suivant:")
        for node in longest_path:
            print(node)
        print("")
    else:
        print("Le graphe contient un cycle\n")
    print("Le chemin le plus court a " + str(len(shortest_path)) + " noeuds et est le suivant:")
    for node in shortest_path:
        print(node)

In [141]:
main('./main_output.dot')

fn_0_basic_block_1:s fn_2_basic_block_4:n
fn_1_basic_block_1:s fn_2_basic_block_3:n
fn_2_basic_block_2:s fn_1_basic_block_0:n
fn_2_basic_block_3:s fn_0_basic_block_0:n
fn_0_basic_block_0:s fn_0_basic_block_2:n
fn_0_basic_block_2:s fn_0_basic_block_3:n
fn_0_basic_block_2:s fn_0_basic_block_4:n
fn_0_basic_block_3:s fn_0_basic_block_5:n
fn_0_basic_block_4:s fn_0_basic_block_5:n
fn_0_basic_block_5:s fn_0_basic_block_1:n
fn_1_basic_block_0:s fn_1_basic_block_2:n
fn_1_basic_block_2:s fn_1_basic_block_3:n
fn_1_basic_block_3:s fn_1_basic_block_1:n
fn_2_basic_block_0:s fn_2_basic_block_2:n
fn_2_basic_block_5:s fn_2_basic_block_7:n
fn_2_basic_block_6:s fn_2_basic_block_7:n
fn_2_basic_block_7:s fn_2_basic_block_8:n
fn_2_basic_block_8:s fn_2_basic_block_1:n
fn_2_basic_block_4:s fn_2_basic_block_5:n
fn_2_basic_block_4:s fn_2_basic_block_6:n
Il y a un noeud sans entrée dans la fonction main

Le graphe est acyclique

Le plus long chemin a 17 noeuds et est le suivant:
fn_2_basic_block_0
fn_2_basic_blo